# Spline curves

General properties:
- polynomial of degree $d$ for $t$
- (distance/projection equation is of degree $2d-1$)
- uniform cooficients for all segments
- locality: $d+1$ points affect a segment, $d-1$ segments affected by a point  
- parametric continuity: at least $C^0$, other depends on arrangement
- geometric continuity can be acheved by arranging control points

## General math

Piece-wise spline curve of degree $d$ with characteristic matrix $M$ of size $(d+1) × (d+1)$

$Q_i(t_l) =
\begin{bmatrix}
1 & t_l & t_l^2 & \dots & t_l^d
\end{bmatrix}
M
\begin{bmatrix}
P_i\\
P_{i+1} \\
\vdots \\
P_{i+d} \\
\end{bmatrix}
$


Each $i$-th segment runs by local $t_l \in [0, 1]$ and global $t = t_i + t_l$. The segment is generally supported by $d+1$ points $P_i, \dots, P_{i+d}$. 

Curve can extrapolate beyond the points with values of $$ outside $[0, 1]$, though. The extrapolation runs with the same speed/acceleration as at an edge point.

### Derivatives


Velocity:

$Q'(t) = \frac{d}{dt} Q(t) = 
\begin{bmatrix}
0 & 1 & 2 t & \dots & d t^{d-1}
\end{bmatrix}
M
\begin{bmatrix}\vdots \\ P \\ \vdots\end{bmatrix}
$

Acceleration:

$Q''(t) = \frac{d^2}{dt^2} Q(t) = 
\begin{bmatrix}
0 & 0 & 2 & \dots & d (d-1) t^{d-2}
\end{bmatrix}
M
\begin{bmatrix}\vdots \\ P \\ \vdots\end{bmatrix}
$

Tangent (normalized):

$T(t) = \frac{Q'(t)}{\|Q'(t)\|}$

Normal (in a plane of the osculating circle):

$N(t) = \frac{d}{dt} T(t) = \frac{Q''(t)}{\|Q''(t)\|}$

In [1]:
import math
import random

import numpy as np
from plotly import graph_objects as go
import ipywidgets as iw
from IPython.display import display

from vectors import vec
import splines as spl

In [2]:
#### generating random points in x-aligned grid

def genpoints2d(n):
    return np.random.random((n, 2))

def genpoints3d(n):
    return np.random.random((n, 3))

# evenly distributed along x

def genpoints2dx(n):
    return np.array([np.arange(n) + np.random.random(n) * 0.5, np.random.random(n)]).T

def genpoints3dx(n):
    return np.array([np.arange(n) + np.random.random(n) * 0.5, np.random.random(n), np.random.random(n)]).T


## Bezier curves

Piece-wise curve, interpolating edge control points, approximating interior control points. 
At its edges the curve is tangent to vectors $P_1 - P_0$ and $P_{d} - P_{d-1}$.

Equivalent to non-uniform B-spline with knot vector of 0 and 1 repeated $d$ times

### Construction

Recursive:

- $Q^{(1)}_i(t|P_0, P_1) = (1-t) P_0 + (t) P_1$ — linear: lerping between 2 points
- $Q^{(2)}_i(t|P_0, P_1, P_2) = (1-t) B^{(1)}(t|P_0, P_1) + (t) B^{(1)}(t|P_1, P_2)$ — cuadratic: lerping between lerped points
- $Q^{(3)}_i(t|P_0, P_1, P_2, P_3) = (1-t) B^{(2)}(t|P_0, P_1, P_2) + (t) B^{(2)}(t|P_1, P_2, P_3)$ — cubic: lerping between lerped^2 points

Polynomial:

$b^{(d)}_k(t) = \binom{d}{k} t^k (1-t)^{(d-k)}$ — Bernstein polynomial

$Q^{(d)}(t) = \sum\limits_{k=0}^{d} b^{(d)}_k(t)P_k$

Segments supproted by generally independent $d+1$ points $P_0 \cdots P_d$, with local $t \in [0, 1]$.

### Smoothness

Continuity depends on construction and relations of the points.

- $C^0$: $P_{i,d} = P_{i+1,0}$ — shared edge point
- $G^1$ (tangent): $\|P_{i,d} - P_{i,d-1}\| = \|P_{i+1,1} - P_{i+1,0}\|$ — edge vectors are aligned with a degreee of freedom 
- $C^1$ (speed): $P_{i,d} - P_{i,d-1} = P_{i+1,1} - P_{i+1,0}$ — edge vectors are mirrored, dependant P_1 and P_{d-1} 
- $G^2, C^2$ (curvature, accel): induces fail of local control (all points dependant)

### Matrices 

$Q^{(2)}(t) =
\begin{bmatrix} 1 & t & t^2 \end{bmatrix}
\begin{bmatrix} 
1 & 0 & 0 \\ 
-2 & 2 & 0 \\ 
1 & -2 & 1 \\ 
\end{bmatrix}
\begin{bmatrix} P_0 \\ P_1 \\ P_2 \end{bmatrix}
$

$Q^{(3)}(t) =
\begin{bmatrix} 1 & t & t^2 & t^3 \end{bmatrix}
\begin{bmatrix} 
1 & 0 & 0 & 0 \\  
-3 & 3 & 0 & 0 \\
3 & -6 & 3 & 0 \\
-1 & 3 & -3 & 1 \\
\end{bmatrix}
\begin{bmatrix} P_0 \\ P_1 \\ P_2 \\ P_3 \end{bmatrix}
$

In [3]:
npoints = 6
points = genpoints2dx(npoints)
tt = np.linspace(0-.125, 1.125, 10)

curve2, _ = spl.build_curve(spl.Bezier2, points[:3])
curve3, _ = spl.build_curve(spl.Bezier3, points[2:6])

pxx, pyy = points.T
c2xx, c2yy = curve2(tt).T
c3xx, c3yy = curve3(tt).T

go.Figure(
    [
        go.Scatter(x=pxx, y=pyy, mode="lines+markers", line=dict(dash="dot"), name="points"),
        go.Scatter(x=c2xx, y=c2yy, mode="lines", name="Bezier 2"),
        go.Scatter(x=c3xx, y=c3yy, mode="lines", name="Bezier 3"),
    ],
    layout=dict(yaxis=dict(range=(0, 1)), xaxis=dict(range=(0, npoints))),
)


## Uniform B-splines

Unlimited overlapping-piecewise curve approximating all control points.

Quadratic B-spline curve: tangent to line segments $[P_{i}, P_{i+1}]$ at midpoints

### Construction

With non-uniform knot vector $T = [t_0, t_1, ..., t_{n+d}]$ dividing curve into $n+d$ segments with any $n$ (points)

$B^{(0)}_j[T](t) =
\begin{cases}
1 \text{ if } t_{j} ≤ t < t_{j+1} \\
0 \text{ otherwise }
\end{cases}$

$B^{(d)}_j[T](t) = \frac{t - t_j}{t_{j+d} - t_j} B^{(d-1)}_j[T](t) + \frac{t_{j+d+1} - t}{t_{j+d+1} - t_{j+1}} B^{(d-1)}_{j+1}[T](t)$

With uniform knot vector $T = t_{j+1} - t_{j} = 1, t_{j} = j$

$B^{(d)}_j(t) = \frac{1}{d} ((t - j) B^{(d-1)}_j(t) + (d + 1 - (t - j)) B^{(d-1)}_{j+1}(t))$

### Smoothness

Continuos everywhere up to $C^d$ on all points.

### Matrices

With normalized $t \in [0,1]$ between points, $j = \lfloor t \rfloor$

$B^{(2)}(t) = 
\begin{bmatrix} 1 & t & t^2 \end{bmatrix}
\frac{1}{2}
\begin{bmatrix} 1 & 1 & 0 \\ -2 & 2 & 0 \\ 1 & -2 & 1 \end{bmatrix}
\begin{bmatrix} P_{j} \\ P_{j+1} \\ P_{j+2} \end{bmatrix}
$

$B^{(3)}(t) =
\begin{bmatrix} 1 & t & t^2 & t^3 \end{bmatrix}
\frac{1}{6}
\begin{bmatrix} 1 & 4 & 1 & 0 \\ -3 & 0 & 3 & 0 \\  3 & -6 & 3 & 0 \\ -1 & 3 & -3 & 1 \end{bmatrix}
\begin{bmatrix} P_{j} \\ P_{j+1} \\ P_{j+2} \\ P_{j+3} \end{bmatrix}
$



In [4]:
npoints = 6
points = genpoints2dx(npoints)

curve2, _ = spl.build_curve(spl.B2, points)
curve3, _ = spl.build_curve(spl.B3, points)

pxx, pyy = points.T
c2xx, c2yy = curve2(np.arange(0, npoints-2, .125)).T
c3xx, c3yy = curve3(np.arange(0, npoints-3, .125)).T

go.Figure(
    [
        go.Scatter(x=pxx, y=pyy, mode="lines+markers", line=dict(dash="dot"), name="points"),
        go.Scatter(x=c2xx, y=c2yy, mode="lines", name="B2"),
        go.Scatter(x=c3xx, y=c3yy, mode="lines", name="B3"),
    ],
    layout=dict(yaxis=dict(range=(0, 1)), xaxis=dict(range=(0, npoints))),
)


# Tangents

$\frac{d}{dt} Q(t)$ — curve velocity

normalized $\frac{d}{dt} Q(t)$ — curve tangent

In [5]:
npoints = 6
points = genpoints2dx(npoints)

M = spl.B2
d = M.shape[0]

tt = np.arange(0, npoints, .125)

curve, curve_dt = spl.build_curve(M, points)

pxx, pyy = points.T
cxx, cyy = curve(tt).T
txx = [0, 0]
tyy = [0, 0]

fig = go.FigureWidget(
    [
        go.Scatter(x=pxx, y=pyy, mode="lines+markers", line=dict(dash="dot"), name="points", hoverinfo="skip"),
        go.Scatter(x=cxx, y=cyy, mode="lines", name="curve", line=dict(width=4), hoverinfo="none"),
        go.Scatter(x=txx, y=tyy, mode="lines+markers", name="curve/dt", marker=dict(size=10, symbol="arrow", angleref="previous"), hoverinfo="skip")
    ],
    layout=dict(yaxis=dict(range=(0, 1)), xaxis=dict(range=(0, npoints))),
)

poins_plt, curve_plt, tang_plt = fig.data

def show_tang(inds):
    if len(inds) == 0:
        tang_plt.visible = False
        return

    i = inds[0]
    tx, ty = curve_dt(tt[i]).T
    with fig.batch_update():
        tang_plt.visible = True
        tang_plt.x = [cxx[i], cxx[i]+tx]
        tang_plt.y = [cyy[i], cyy[i]+ty]


fig.data[1].on_hover(lambda _t, pnts, _d: show_tang(pnts.point_inds))
display(fig)

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'dash': 'dot'},
              'mode': 'lines+markers',
              'name': 'points',
              'type': 'scatter',
              'uid': 'c90e4528-cf16-43d8-b6ee-485a87d06d98',
              'x': array([0.07285704, 1.44445695, 2.38565707, 3.24162678, 4.05987696, 5.07060702]),
              'y': array([0.38614553, 0.71318799, 0.32734232, 0.84736244, 0.45062323, 0.28447209])},
             {'hoverinfo': 'none',
              'line': {'width': 4},
              'mode': 'lines',
              'name': 'curve',
              'type': 'scatter',
              'uid': '50d7216e-4941-4a40-9706-78848c9874bf',
              'x': array([0.758657  , 0.92674449, 1.08810698, 1.24274448, 1.39065698, 1.53184448,
                          1.66630699, 1.7940445 , 1.91505701, 2.03204116, 2.14769359, 2.26201429,
                          2.37500327, 2.48666052, 2.59698605, 2.70597985, 2.81364192, 2.92034345,
                        

# Projection

Finding closest point $p^* = q(t^*)$ on curve.

Distance to curve squared:

$\|(p - p^*)\|^2 = (p - p^*) \cdot (p - p^*)$

Signed distance:

$d^* = 
\begin{cases} + \|p - p^*\| & \text{ if (rightward of tangent) } & [(p - p^*) × T(t^*)]_z > 0 \\ - \|p - p^*\| & \text{ if (leftward of tangent) }  & [(p - p^*) × T(t^*)]_z < 0 \end{cases}
$

General equation of closest point:

$(p - q(t^*)) \cdot q'(t^*) = 0$

The equation is of degree $2d-1$.

#### Problems

- generally solvable only for degrees below 3 (for linear and quadratic curves)
- multiple legit solutions possible for points inside a lobe, including points with exactly the same distance  



## Linear case

$q(t) = p_0 + (p1 - p0) t$

$q'(t) = p1 - p0$

Equation:

$(p - q(t^*))·q'(t^*) = 0$

$t^* = \frac{(p - p_0)·(p_1 - p_0)}{(p_1 - p_0)·(p_1 - p_0)}$


In [6]:
def project_1(p0, p1):
    q1 = p1 - p0

    def proj(p):
        return ((p - p0) @ q1)/(q1 @ q1)

    return proj

## Quadratic

$
q(t) = 
\begin{bmatrix} 1 & t & t^2 \end{bmatrix}
\begin{bmatrix} q_0 \\ q_1 \\ q_2 \end{bmatrix} = 
q_0 + q_1 t + q_2 t^2 
$

$
q'(t) =
\begin{bmatrix} 0 & 1 & 2t \end{bmatrix}
\begin{bmatrix} q_0 \\ q_1 \\ q_2 \end{bmatrix} = 
q_1 + 2 q_2 t
$


$q_p = p - q_0$

Equation:

$(p - q_0 - q_1 t - q_2 t^2) \cdot (q_1 + 2 q_2 t) = 0$

$
\begin{bmatrix} 1 & t & t^2 & t^3 \end{bmatrix}
\begin{bmatrix}
(q_p \cdot q_1) \\
2 (q_p \cdot q_2) - (q_1 \cdot q_1) \\
-3 (q_1 \cdot q_2) \\
-2 (q_2 \cdot q_2) \\
\end{bmatrix}
=0
$

In [7]:
### solving by numpy for general curves

def project_2np(M, cpoints):
    """returns all possible solutions """
    Q = M @ cpoints

    def proj(p):
        qp = p - Q[0]
        poly = np.polynomial.Polynomial([
            np.dot(qp,Q[1]),
            2*np.dot(qp, Q[2]) - np.dot(Q[1], Q[1]),
            -3*np.dot(Q[1], Q[2]),
            -2*np.dot(Q[2], Q[2])
        ])
        roots = poly.roots()
        return roots[np.isreal(roots)].real

    return proj

In [8]:
def project_B2_lin(curve):
    """projecting via linear approximation between midpoints"""
    pm0 = curve(0)
    pm = curve(0.5)
    pm1 = curve(1)

    proj0 = project_1(pm0, pm)
    proj1 = project_1(pm, pm1)

    def proj(p):
        t = proj1(p)
        if t > 0:
            return np.array([t * 0.5 + 0.5])

        t = proj0(p)
        if t < 1:
            return np.array([t * 0.5])

        return np.array([0.5])

    return proj

In [9]:
def distmap(curve, tang, proj):

    @np.vectorize(signature="(),()->(),()")
    def dist(x, y):
        "plane (x,y) -> (t*, dist)"
        p = vec(x, y)
        tt = proj(p)
        jj = p - curve(tt)
        ll = (jj * jj).sum(1)

        lmin = np.min(ll)
        ismin = np.isclose(ll, lmin)

        t_best = tt[ismin][0]
        j_best = jj[ismin][0]
        g_best = tang(t_best)

        dist = math.sqrt(lmin)
        z_cross = j_best[0] * g_best[1] - j_best[1] * g_best[0]
        sdist = dist if z_cross > 0 else -dist

        return t_best, sdist

    return dist

In [10]:
npoints = 3
points = genpoints2dx(npoints)
# points = np.array([vec(0, 0), vec(1.0, 1.0), vec(2, 0)])

curve, curve_dt = spl.build_curve(spl.B2, points)
# proj = project_2np(spl.B2, points)
proj = project_B2_lin(curve)

pxx, pyy = points.T

tt = np.linspace(0, 1, 16)
cxx, cyy = curve(tt).T

range_x = np.min(cxx), np.max(cxx)
range_y = np.min(cyy), np.max(cyy)

uu = np.arange(*range_x, 1 / 64)
vv = np.arange(*range_y, 1 / 64)

uuu, vvv = np.meshgrid(uu, vv)

tproj, dist = distmap(curve, curve_dt, proj)(uuu, vvv)

txx = [0, 0]
tyy = [0, 0]

fig = go.FigureWidget(
    [
        go.Scatter(x=cxx, y=cyy, mode="lines", name="curve", line=dict(width=4), hoverinfo="none"),
        go.Heatmap(
            x=uu,
            y=vv,
            z=dist,
            name="distmap",
            hovertemplate="x: %{x:.2f}<br>y: %{y:.2f}<br>dist: %{z:.3f}",
            colorscale="RdBu_r",
        ),
        go.Scatter(x=txx, y=tyy, mode="markers", name="proj", marker=dict(size=10, color="black"), hoverinfo="skip", visible=False),
    ],
    layout=go.Layout(xaxis=dict(scaleanchor="y"), showlegend=False),
)

curve_plt, dist_plt, proj_plt = fig.data


def show_proj(inds):
    if len(inds) == 0:
        proj_plt.visible = False
        return

    i, j = inds[0]
    px = uuu[i][j]
    py = vvv[i][j]
    tt = proj(vec(px, py))
    cx, cy = curve(tt).T
    with fig.batch_update():
        proj_plt.visible = True
        proj_plt.x = cx
        proj_plt.y = cy


dist_plt.on_hover(lambda _t, pnts, _d: show_proj(pnts.point_inds))

display(fig)

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'width': 4},
              'mode': 'lines',
              'name': 'curve',
              'type': 'scatter',
              'uid': 'f1502078-6bbc-46b2-9247-584aa7f7a430',
              'x': array([0.83954691, 0.9116883 , 0.98335739, 1.05455418, 1.12527866, 1.19553084,
                          1.26531071, 1.33461828, 1.40345355, 1.47181651, 1.53970716, 1.60712552,
                          1.67407157, 1.74054531, 1.80654675, 1.87207589]),
              'y': array([0.51474729, 0.56923821, 0.62013062, 0.66742453, 0.71111993, 0.75121682,
                          0.7877152 , 0.82061508, 0.84991645, 0.87561932, 0.89772368, 0.91622953,
                          0.93113687, 0.94244571, 0.95015604, 0.95426787])},
             {'colorscale': [[0.0, 'rgb(5,48,97)'], [0.1, 'rgb(33,102,172)'],
                             [0.2, 'rgb(67,147,195)'], [0.3, 'rgb(146,197,222)'],
                             [0.4, 'rgb(209,229,240)'

# Pipe surface

Directrix (spine): 
- point: $A(u) \in R^3$ 
- tangent: $\frac{d}{du}A(u) \in R^3$

Generatrix (profile): $C(v) \in R^2$

Circle (measuring $\phi$ from top): 

$C(\phi) = r \begin{bmatrix} sin(\phi) \\ cos(\phi) \end{bmatrix}$

$C_{norm}(\phi) = \begin{bmatrix} sin(\phi) \\ cos(\phi) \end{bmatrix}$

Generatrix frame aligned to horizon:

- $z$-axis: $T(u) \simeq A'(u)$ — spine tangent
- $x$-axis: $X(u) \simeq  T(u) × e_z$ — horizontal and perpendicular to tangent
- $y$-axis: $Y(u) \simeq  X(u) × T(u)$ — mutually perpendicular and approximately up

(everything should be normalized)

Pipe surface, with $v = \phi$:

$S(u, v) = A(u) + C(v)_x X(u) + C(v)_y Y(u)$

Normal to the surface:

$N(u, v) = \begin{bmatrix} X(u) \\ Y(u) \\ T(u) \end{bmatrix}^T C_{norm}(v)$ — translated into uv fabric frame

In [17]:
import pipes

In [18]:
# test texture
def checker(u, v):
    "(u, v) -> rgb"
    val = np.mod(np.floor(u) + np.floor(v), 2)
    return val

In [19]:
npoints = 6
points = genpoints3dx(npoints)

M = spl.B2
d = M.shape[0]

curve, curve_dt = spl.build_curve(M, points)
pipe, pipe_norm = pipes.build_pipe(curve, curve_dt, 0.25)

t1 = npoints - d + 1
tt = np.arange(0, t1, 1/8)
ss = np.linspace(-0.5 * math.pi, +0.5 * math.pi, 13)
uu, vv = np.meshgrid(tt, ss)

pxx, pyy, pzz = points.T
cxx, cyy, czz = curve(tt).T
xx, yy, zz = pipe(uu, vv).T
nxx, nyy, nzz = pipe_norm(uu, vv).T

tex = checker(uu, vv / math.pi).T

go.Figure(
    [
        go.Scatter3d(x=pxx, y=pyy, z=pzz, mode="lines+markers", line=dict(dash="dot"), name="points", hoverinfo="skip"),
        go.Scatter3d(x=cxx, y=cyy, z=czz, mode="lines", line=dict(width=4), name="curve", hoverinfo="skip"),
        go.Surface(x=xx, y=yy, z=zz, surfacecolor=tex, hoverinfo="skip", colorscale=["black","white"], showscale=False),
        go.Cone(x=xx.flatten(), y=yy.flatten(), z=zz.flatten(), u=nxx.flatten(), v=nyy.flatten(), w=nzz.flatten(), showscale=False, hoverinfo="skip"),
    ],
    layout=go.Layout(scene=dict(aspectmode="data", aspectratio={"x": 1, "y": 1, "z": 1}), height=512),
)

# Texture mapping

Mapping of fabric plane points $(u, v)$ into pipe surface $(t, \phi)$ (with the $t$ approximating arc length)

- $p = [u, v]$ ­— point on the UV fabric plane in locality of a curve segment
- $p^* = q(t^*)$ — closest point on the curve
- $s^*$ — signed distance from the $p$ to $p^*$ 
- $X(t^*), Y(t^*), Z(t^*)$ — local generatrix frame at the closest point

In the local frame with semi-circle profile with $\phi$ measured from top point, negative to left, positive to right:

$x = s^* = r sin(\phi^*), y = r cos(\phi^*)$

$\phi^* = 
\begin{cases}
arcsin(\frac{s^*}{r}) & \text { if } |s^*| < r  \\
\inf & \text{ otherwise }
\end{cases}
$

For low fidelity renders (flat mapping), $arcsin(a) \approx a \frac{\pi}{2}$

For texture mapping in range $[0,1]$, $v = 0.5 + \frac{\phi}{\pi}$

In [20]:
def vwrap_flat(r):

    @np.vectorize
    def wrap(s):
        if abs(s) <= r:
            return s * 0.5 * math.pi
        else:
            return np.nan

    return wrap


def vwrap_sin(r):

    @np.vectorize
    def wrap(s):
        if abs(s) <= r:
            return math.asin(s / r)
        else:
            return np.nan

    return wrap

In [21]:
def wrap3d(fn):

    @np.vectorize(signature="(),()->(3)")
    def wrapped(u, v):
        if np.isnan(u) or np.isnan(v):
            return np.array([np.nan, np.nan, np.nan])
        else:
            return fn(u, v)

    return wrapped


@np.vectorize(signature="(3)->(3)")
def vec2rgb(p):
    if np.isnan(p[0]):
        return np.array([math.nan, math.nan, math.nan])
    return p * 256

@np.vectorize(signature="()->(3)")
def val2rgb(v):
    if np.isnan(v):
        return np.array([math.nan, math.nan, math.nan])
    b = v * 256
    return np.array([b, b, b])


In [23]:
points = genpoints3d(3)
points2d = points[:, :2]

# flatten a bit
points[0][2] = 0.5
points[1][2] = 0.5 + 0.5 * points[1][2]
points[2][2] = 0.5

thickness = 0.25 / 4

#### 3d model

curve3, curve3_dt = spl.build_curve(spl.B2, points)
curve2, curve2_dt = spl.build_curve(spl.B2, points2d)
proj2 = project_B2_lin(curve2)
pipe, pipe_norm = pipes.build_pipe(curve3, curve3_dt, thickness)

pxx, pyy, pzz = points.T
cxx, cyy, czz = curve3(np.arange(0, 1, 1 / 8)).T
suu, svv = np.meshgrid(np.arange(0, 1, 1 / 8), np.linspace(-0.5 * math.pi, +0.5 * math.pi, 7))
sxx, syy, szz = pipe(suu, svv).T
nxx, nyy, nzz = pipe_norm(suu, svv).T
stex = checker(suu * 4, svv / math.pi).T

#### projection

range_x = np.min(cxx) - thickness, np.max(cxx) + thickness
range_y = np.min(cyy) - thickness, np.max(cyy) + thickness

pixelsize = 1 / 128

xx = np.arange(*range_x, pixelsize)
yy = np.arange(*range_y, pixelsize)
img_range = dict(x0=range_x[0], dx=pixelsize, y0=range_y[0], dy=pixelsize)

xxx, yyy = np.meshgrid(xx, yy)

# [x][y]
tproj, sdist = distmap(curve2, curve2_dt, proj2)(xxx, yyy)
uuu = tproj
vvv = vwrap_sin(thickness)(sdist)

tex = val2rgb(checker(uuu * 4, vvv / math.pi))

surface = pipe(uuu, vvv)
tex_z = val2rgb(surface[:, :, 2])

normals = pipe_norm(uuu, vvv)
tex_n = vec2rgb(normals * vec(0.5, 0.5, 1) + vec(0.5, 0.5, 0))


fig3d = go.FigureWidget(
    [
        go.Scatter3d(x=cxx, y=cyy, z=czz, mode="lines", name="curve", line=dict(width=4), hoverinfo="skip"),
        go.Scatter3d(x=pxx, y=pyy, z=pzz, mode="lines+markers", line=dict(dash="dot"), name="points", hoverinfo="skip"),
        go.Surface(x=sxx, y=syy, z=szz, surfacecolor=stex, colorscale=["black", "white"], hoverinfo="skip", showscale=False),
        go.Cone(x=sxx.flatten(), y=syy.flatten(), z=szz.flatten(), u=nxx.flatten(), v=nyy.flatten(), w=nzz.flatten(), showscale=False, hoverinfo="skip"),
    ],
    layout=go.Layout(
        scene=dict(
            camera=dict(eye=dict(x=0, y=0, z=5), center=dict(x=0, y=0, z=0), up=dict(x=0, y=-1, z=0)),
            aspectmode="data",
            aspectratio=dict(x=1, y=1, z=1),
            xaxis_autorange="reversed",
            yaxis_autorange="reversed",
        ),
        margin=dict(l=8, r=8, b=8),
        title="Local segment of curve in 3d",
    ),
)

fig2tex = go.FigureWidget(
    [
        go.Image(z=tex, **img_range),
    ],
    layout=go.Layout(
        xaxis=dict(range=(0, 1)),
        yaxis=dict(range=(0, 1)),
        showlegend=False,
        margin=dict(l=8, r=8, b=8),
        title="Projection of texture to XY",
    ),
)

fig2z = go.FigureWidget(
    [
        go.Image(z=tex_z, **img_range),
    ],
    layout=go.Layout(
        xaxis=dict(range=(0, 1)),
        yaxis=dict(range=(0, 1)),
        showlegend=False,
        margin=dict(l=8, r=8, b=8),
        title="Projection of bump to XY",
    ),
)

fig2n = go.FigureWidget(
    [
        go.Image(z=tex_n, **img_range),
    ],
    layout=go.Layout(
        xaxis=dict(range=(0, 1)),
        yaxis=dict(range=(0, 1)),
        showlegend=False,
        margin=dict(l=8, r=8, b=8),
        title="Projection of normal to XY",
    ),
)

display(iw.HBox([fig3d, iw.VBox([fig2tex, fig2z, fig2n])]))

    'data': [{'hoverinfo': 'skip',
              'line': {'width': 4},
         …